## sort

1. `List.sort` 和 `Collections.sort`(内部调用前者) 都是原地方法;
2. `List.of / Map.of` 等等创建的不可变集合不支持修改故不支持 sort, 复制新的再进行或者使用 `.stream().sorted`;
3. `thenComparing` 前的 `Comparator.comparingXXX` 必须要类型转换, 否则无法类型推断

### 1. list

In [21]:
var nameOrder = List.of("Alice", "Bob", "Bruce");
var persons = List.of(
    Map.of("name", "Bruce", "age", "2"),
    Map.of("name", "Alice", "age", "2", "require", "true"),
    Map.of("name", "Bob", "age", "1")
);
persons

persons: [{name=Bruce, age=2}, {age=2, name=Alice, require=true}, {name=Bob, age=1}]

#### 1.1 simple comparator

`List.sort` 和 `Collections.sort` 都是原地方法, 需要生成新的可以使用 `.stream().sorted`

In [28]:
import java.util.stream.IntStream;
import java.util.stream.Collectors;

var idsIdxMap = IntStream.range(0, nameOrder.size()).boxed().collect(Collectors.toMap(i -> nameOrder.get(i), i -> i));
idsIdxMap

idsIdxMap: {Bruce=2, Bob=1, Alice=0}

In [35]:
var p2 = new ArrayList<>(persons);

p2.sort(Comparator.comparingInt(m -> idsIdxMap.get(m.get("name"))));
print(p2);

persons.stream().sorted(Comparator.comparingInt(m -> idsIdxMap.get(m.get("name")))).toList()

p2: [{age=2, name=Alice, require=true}, {name=Bob, age=1}, {name=Bruce, age=2}]


persons.stream().sorted(Comparat...: [{age=2, name=Alice, require=true}, {name=Bob, age=1}, {name=Bruce, age=2}]

#### 1.2 chain comparator

In [37]:
persons.stream()
    .sorted(Comparator.comparing((Map<String, String> m) -> m.get("age"), Comparator.reverseOrder())
            .thenComparing(m -> m.get("require"), Comparator.nullsLast(Comparator.naturalOrder())))
    .toList()

persons.stream().sorted(Comparat...: [{age=2, name=Alice, require=true}, {name=Bruce, age=2}, {name=Bob, age=1}]

#### 1.3 自定义 comparator

In [32]:
// first by length
Comparator<String> stringComp = (a, b) -> a.length() == b.length() ? a.compareTo(b) : Integer.compare(a.length(), b.length());

In [39]:
persons.stream().sorted(((o1, o2) -> stringComp.compare(o1.get("name"), o2.get("name"))));

persons.stream().sorted(((o1,o2)...: java.util.stream.SortedOps$OfRef@5ed5f78e

### 2. map

In [3]:
var map = Map.of("name", "Alice", "age", "2", "require", "true");
map

map: {age=2, name=Alice, require=true}

In [4]:
Map<String, String> result = new LinkedHashMap<>();

map.entrySet().stream()
    .sorted(Map.Entry.comparingByKey())
    .forEachOrdered(x -> result.put(x.getKey(), x.getValue()));

result

result: {age=2, name=Alice, require=true}

### 3. stream

In [2]:
List.of(1, 2, 3, -1, -2)
    .stream()
    .sorted(Comparator.comparingInt(i -> i))
    .toList();

List.of(1,2,3,-1,-2).stream().so...: [-2, -1, 1, 2, 3]

## modify while iterating: `Iterator/ListIterator`

1. `Iterator.remove`;
2. `List.listIterator` 生成的是 `ListIterator`, 其额外支持 `add`

In [2]:
List<Integer> list = new ArrayList<>(List.of(1, 2, 3));
// list.removeIf(num -> num % 2 == 0);
for (Iterator<Integer> it = list.iterator(); it.hasNext() ;) {
    var num = it.next();
    if (num % 2 == 0) {
        it.remove();
    }
}
System.out.println("list: " + list);

for (ListIterator<Integer> it = list.listIterator(); it.hasNext() ;) {
    var num = it.next();
    if (num % 2 == 0) {
        it.remove();
    } else if (num % 3 == 0) {
        it.add(3);
        it.add(30);
    }
}
System.out.println("list: " + list);

list: [1, 3]
list: [1, 3, 3, 30]
